• train(): where you build your Machine Learning model from the training data.
• predict(): where you use your trained model to predict a class for the validation data.
• evaluate(): where you will output the accuracy of your model based on an evaluation metric.

In [50]:
import pandas as pd

def load_csvdata(csvFilename):
    data = pd.read_csv(open(csvFilename))
    return data

def preprocess(data):
    data.drop('trackID', axis = 1, inplace = True) #axis ?, inplace?
    data.rename(columns = {'key': 'key_cat'}, inplace = True) # == 
    
from sklearn.feature_extraction.text import TfidfVectorizer
def TFIDF(column, dfmin = 1, dfmax = 0.5):
    vectorizer = TfidfVectorizer(stop_words='english', min_df = dfmin, max_df = dfmax)
    train = vectorizer.fit_transform(train_F[column].values.astype('U'))
    train_TFIDF = pd.DataFrame(train.toarray())
    words = vectorizer.get_feature_names()
    train_TFIDF.columns = words
    
    valid = vectorizer.transform(valid_F[column].values.astype('U'))
    valid_TFIDF = pd.DataFrame(valid.toarray())
    valid_TFIDF.columns = words

    test = vectorizer.transform(test_F[column].values.astype('U'))
    test_TFIDF = pd.DataFrame(test.toarray())
    test_TFIDF.columns = words
    return train_TFIDF, valid_TFIDF, test_TFIDF

def normalization():
    #store the columns should be droped. If the values of one column are same in every instances, this column is useless.
    #droplist = []
    for i in range(len(train_final.columns)): #Min-Max Normalization or Standard score.
        
        if i in range(2 ,5):
            continue
        
        max = train_final.iloc[:,i].max()
        min = train_final.iloc[:,i].min()
        
        #if max == min:
         #   droplist.append(train_data.iloc[:,i].name)
          #  continue
        #The features must process same opeartion 
        train_final.iloc[:,i] = (train_final.iloc[:,i] - min)/(max - min)
        valid_final.iloc[:,i] = (valid_final.iloc[:,i] - min)/(max - min)
        test_final.iloc[:,i] = (test_final.iloc[:,i] - min)/(max - min)
        
    #for i in droplist:
        #The features must process same opeartion 
     #   train_final.drop(i, axis = 1, inplace = True)
      #  valid_final.drop(i, axis = 1, inplace = True)
       # test_final.drop(i, axis = 1, inplace = True)

def final_input(data, title_TFIDF, tag_TFIDF):
    
    inputdata = pd.concat([data, title_TFIDF, tag_TFIDF], axis=1)
    #since we take the key words in title and tag, the raw data is useless now.
    inputdata.drop('title', axis = 1, inplace = True)
    inputdata.drop('tags', axis = 1, inplace = True)
    return inputdata

In [60]:
train_F = load_csvdata("train_features.csv")
train_L = load_csvdata("train_labels.csv")
valid_F = load_csvdata("valid_features.csv")
valid_L = load_csvdata("valid_labels.csv")
test_F = load_csvdata("test_features.csv")

preprocess(train_F)
preprocess(train_L)
preprocess(valid_F)
preprocess(valid_L)
preprocess(test_F)

train_title_TFIDF, valid_title_TFIDF, test_title_TFIDF = TFIDF('title', 9, 0.3)
train_tag_TFIDF, valid_tag_TFIDF, test_tag_TFIDF = TFIDF('tags')

train_final = final_input(train_F, train_title_TFIDF, train_tag_TFIDF)
valid_final = final_input(valid_F, valid_title_TFIDF, valid_tag_TFIDF)
test_final = final_input(test_F, test_title_TFIDF, test_tag_TFIDF)

normalization()

In [63]:
pd.set_option('max_columns',1000) 
pd.set_option('max_row',300)
train_final.corr()

loudness     tempo  time_signature   key_cat      mode  \
loudness        1.000000  0.214292        0.065769  0.043003 -0.082034   
tempo           0.214292  1.000000       -0.012236  0.007539 -0.020649   
time_signature  0.065769 -0.012236        1.000000 -0.007444 -0.036339   
key_cat         0.043003  0.007539       -0.007444  1.000000 -0.170470   
mode           -0.082034 -0.020649       -0.036339 -0.170470  1.000000   
...                  ...       ...             ...       ...       ...   
ça             -0.009725  0.002934       -0.009027  0.004503 -0.011832   
él              0.019312  0.001083       -0.000495 -0.013111 -0.014627   
était          -0.038678 -0.004979       -0.001411  0.000362 -0.018508   
être           -0.016454  0.011455       -0.018860  0.011283 -0.000435   
über            0.025172  0.019281        0.004654  0.008802  0.014562   

                duration    vect_1    vect_2    vect_3    vect_4    vect_5  \
loudness        0.033833  0.945383  0.589115  0.250068  0.239312 -0.282216   
tempo          -0.041976  0.252205  0.213809  0.072412  0.076023 -0.188187   
time_signature  0.110843  0.029111  0.020796 -0.045944  0.020493  0.041607   
key_cat         0.030415  0.032304  0.035447 -0.009544  0.031644 -0.023010   
mode           -0.102387 -0.057502 -0.065407  0.022726 -0.081160  0.017713   
...                  ...       ...       ...       ...       ...       ...   
ça             -0.002634 -0.010146 -0.000281 -0.006572  0.005013 -0.001370   
él             -0.007739  0.014581  0.001640  0.009855 -0.003736  0.016513   
était           0.000039 -0.036443 -0.026798 -0.024677 -0.020328  0.010878   
être            0.000158 -0.017338 -0.005332 -0.019451 -0.006974  0.017102   
über            0.008724  0.025333  0.020530 -0.003699 -0.005870 -0.015247   

                  vect_6    vect_7    vect_8    vect_9   vect_10   vect_11  \
loudness       -0.324373  0.274825  0.075629  0.212698  0.305488  0.122580   
tempo          -0.159040  0.103258  0.036679  0.102198  0.172033  0.068885   
time_signature -0.046988  0.024917 -0.062894 -0.000693 -0.010509  0.030293   
key_cat         0.010296  0.031500  0.031643 -0.018884  0.022233  0.016488   
mode            0.031730 -0.059893 -0.033017 -0.001791 -0.075401 -0.039290   
...                  ...       ...       ...       ...       ...       ...   
ça              0.000237  0.005714  0.006418 -0.010461 -0.001349  0.012963   
él              0.007256 -0.019890 -0.000220  0.006407  0.000621 -0.001552   
était          -0.000211  0.003679 -0.003518  0.005082 -0.012416 -0.008310   
être           -0.003683 -0.019922  0.000839  0.001120  0.007806  0.017269   
über            0.004088 -0.008571 -0.002437 -0.001364 -0.008279 -0.003597   

                 vect_12   vect_13   vect_14   vect_15   vect_16   vect_17  \
loudness       -0.181467 -0.217394 -0.364637 -0.305269 -0.252602 -0.306250   
tempo          -0.057816 -0.163491 -0.210952 -0.182159 -0.183460 -0.186885   
time_signature -0.030884  0.066487  0.039011  0.046788  0.014585  0.054403   
key_cat        -0.017708 -0.010509 -0.010204  0.007564  0.004347 -0.007238   
mode            0.039368 -0.004697 -0.007369 -0.042728 -0.016629 -0.021262   
...                  ...       ...       ...       ...       ...       ...   
ça              0.023709  0.000458  0.000774  0.003160  0.000180 -0.002225   
él             -0.003696  0.004433 -0.001285 -0.002909 -0.010675 -0.008756   
était           0.018371 -0.000807  0.007815  0.003858  0.005066  0.016888   
être            0.027048 -0.007613  0.008146 -0.002248  0.007618  0.003000   
über            0.001127 -0.011809 -0.012301 -0.012082 -0.011030 -0.013841   

                 vect_18   vect_19   vect_20   vect_21   vect_22   vect_23  \
loudness       -0.165850 -0.371515 -0.291607 -0.449760 -0.306023 -0.186178   
tempo          -0.112392 -0.206965 -0.175227 -0.214576 -0.180263 -0.152077   
time_signature  0.018749  0.035777  0.029374  0.010393  0.023238  0.025562   
key_

In [52]:
"""
def DeTree(train_X, train_Y): 
    from sklearn import tree
    Dtree = tree.DecisionTreeClassifier(criterion='gini') #entropy
    Dtree.fit(train_X, train_L.values.ravel())
    return Dtree

def RandomF(train_X, train_Y):
    from sklearn.ensemble import RandomForestClassifier
    Rfc = RandomForestClassifier(criterion='entropy')
    Rfc.fit(train_X, train_Y.values.ravel())
    return Rfc

def KNN(train_X, train_Y, num_neighbors):
    from sklearn.neighbors import KNeighborsClassifier
    knn = KNeighborsClassifier(n_neighbors = num_neighbors)# 71, 61, 51, 81, 91
    knn.fit(train_X, train_Y.values.ravel())
    return knn
"""   

def MNB(train_X, train_Y):
    from sklearn.naive_bayes import MultinomialNB
    mnb = MultinomialNB()
    mnb.fit(train_X, train_Y.values.ravel())
    return mnb

def BNB(train_X, train_Y):
    from sklearn.naive_bayes import BernoulliNB
    bnb = BernoulliNB()
    bnb.fit(train_X, train_Y.values.ravel())
    return bnb

def GNB(train_X, train_Y):
    from sklearn.naive_bayes import GaussianNB
    gnb = GaussianNB()
    gnb.fit(train_X, train_Y.values.ravel())
    return gnb

def SVM(train_X, train_Y):
    from sklearn import svm
    #svc = svm.SVC(kernel='linear')# linear in OK for this problem, but I tried svm.NuSVC(gamma='auto')
    svc = svm.SVC(kernel= 'rbf', random_state = 0)
    svc.fit(train_X, train_Y.values.ravel())
    return svc

In [53]:
mnb = MNB(train_final, train_L)

In [54]:
# This function should predict the class for an instance or a set of instances, based on a trained model
def predict(model, predict_X):
    return model.predict(predict_X)

# This function should evaluate a set of predictions in terms of accuracy
def evaluate(predict_real, predict_Y):
    from sklearn.metrics import accuracy_score
    return accuracy_score(predict_real, predict_Y)

In [55]:
#predict_Y_knn = predict(knn, valid_final)
predict_Y_mnb = predict(mnb, valid_final)
#predict_Y_bnb = predict(bnb, valid_final)
#predict_Y_gnb = predict(gnb, valid_final)
#predict_Y_svm = predict(svm, valid_final)
#predict_Y_detree = predict(detree, valid_final)
#predict_Y_randomf = predict(randomf, valid_final)

#print(evaluate(valid_L ,predict_Y_knn))
print(evaluate(valid_L ,predict_Y_mnb))
#print(evaluate(valid_L ,predict_Y_bnb))
#print(evaluate(valid_L ,predict_Y_gnb))
#print(evaluate(valid_L ,predict_Y_svm))
#print(evaluate(valid_L ,predict_Y_detree))
#print(evaluate(valid_L ,predict_Y_randomf))

0.6511111111111111


In [48]:
# download the prediction of test and output csv file.
def test_submit_files(model):
    predict_test = predict(model, test_final)
    trackID = load_csvdata("test_features.csv")['trackID']
    submit = pd.concat([pd.DataFrame(trackID), pd.DataFrame(predict_test)], axis=1)
    submit.columns = ['trackID', 'genre']
    submit.to_csv(r'submit.csv', index = False, header=True)

In [49]:
test_submit_files(mnb)